# Tunable Baseline Bot

In [ ]:
### This is a fork of https://www.kaggle.com/david1013/tunable-baseline-bot


##  Any upvotes below to his original notebook.

### Tunable Parameters

In [ ]:
%%writefile submission.py

## TUNABLE BASELINE BOT

# Tune Here:
SPRINT_RANGE = 0.6

SHOT_RANGE_X = 0.7  
SHOT_RANGE_Y = 0.2

GOALIE_OUT = 0.2
LONG_SHOT_X = 0.4
LONG_SHOT_Y = 0.2


### Install

In [ ]:
# Install:
# Kaggle environments.
!git clone --quiet https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install -q .

# GFootball environment.
!apt-get -qq update -y
!apt-get -qq install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone --quiet -b v2.3 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget -q https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.3.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install -q .


### Agent

In [ ]:
%%writefile -a submission.py

from kaggle_environments.envs.football.helpers import *
from math import sqrt

directions = [
[Action.TopLeft, Action.Top, Action.TopRight],
[Action.Left, Action.Idle, Action.Right],
[Action.BottomLeft, Action.Bottom, Action.BottomRight]]

dirsign = lambda x: 1 if abs(x) < 0.01 else (0 if x < 0 else 2)

enemyGoal = [1, 0]
GOALKEEPER = 0


In [ ]:
%%writefile -a submission.py

shot_range = [[SHOT_RANGE_X, 1], [-SHOT_RANGE_Y, SHOT_RANGE_Y]]


In [ ]:
%%writefile -a submission.py

def inside(pos, area):
    return area[0][0] <= pos[0] <= area[0][1] and area[1][0] <= pos[1] <= area[1][1]

In [ ]:
%%writefile -a submission.py

@human_readable_agent
def agent(obs):
    controlled_player_pos = obs['left_team'][obs['active']]
    
    if obs["game_mode"] == GameMode.Penalty:
        return Action.Shot
    if obs["game_mode"] == GameMode.Corner:
        if controlled_player_pos[0] > 0:
            return Action.Shot
    if obs["game_mode"] == GameMode.FreeKick:
        return Action.Shot
    
    # Make sure player is running down the field.
    if  0 < controlled_player_pos[0] < SPRINT_RANGE and Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    elif SPRINT_RANGE < controlled_player_pos[0] and Action.Sprint in obs['sticky_actions']:
        return Action.ReleaseSprint

    # If our player controls the ball:
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        
        if inside(controlled_player_pos, shot_range) and controlled_player_pos[0] < obs['ball'][0]:
            return Action.Shot
        
        elif ( abs(obs['right_team'][GOALKEEPER][0] - 1) > GOALIE_OUT   
                and controlled_player_pos[0] > LONG_SHOT_X and abs(controlled_player_pos[1]) < LONG_SHOT_Y ):
            return Action.Shot
        
        else:
            xdir = dirsign(enemyGoal[0] - controlled_player_pos[0])
            ydir = dirsign(enemyGoal[1] - controlled_player_pos[1])
            return directions[ydir][xdir]
        
    # if we we do not have the ball:
    else:
        # Run towards the ball.
        xdir = dirsign(obs['ball'][0] - controlled_player_pos[0])
        ydir = dirsign(obs['ball'][1] - controlled_player_pos[1])
        return directions[ydir][xdir]

In [ ]:
#Test the agent

In [ ]:
import os
import sys 
sys.path.append(os.path.abspath("/kaggle_simulations/agent/"))
import submission

In [ ]:
from kaggle_environments import make
env = make("football", 
           debug=True,
           configuration={"save_video": True, 
                          "scenario_name": "11_vs_11_kaggle", 
                          "running_in_notebook": True,
                          #"actTimeout": 30,
                         })  
output = env.run([submission.agent, "/kaggle_simulations/agent/submission.py"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)